In [ ]:
import pickle
import os
import pandas as pd

In [ ]:
if not os.path.exists('secret_twitter_credentials.pkl'):
    Twitter={}
    Twitter['Consumer Key'] = ''
    Twitter['Consumer Secret'] = ''
    Twitter['Access Token'] = ''
    Twitter['Access Token Secret'] = ''
    with open('secret_twitter_credentials.pkl','wb') as f:
        pickle.dump(Twitter, f)
else:
    Twitter=pickle.load(open('secret_twitter_credentials.pkl','rb'))

In [ ]:
!pip install twitter

In [ ]:
import twitter
auth = twitter.oauth.OAuth(Twitter['Access Token'], Twitter['Access Token Secret'], Twitter['Consumer Key'], Twitter['Consumer Secret'])
twitter_api = twitter.Twitter(auth=auth)
print(twitter_api)

In [ ]:
WORLD_WOE_ID = 1
US_ID = 23424977
LOCAL_WOE_ID = 2295424

In [ ]:
world_trends = twitter_api.trends.place(_id=WORLD_WOE_ID)
us_trends = twitter_api.trends.place(_id=US_ID)
local_trends = twitter_api.trends.place(_id=LOCAL_WOE_ID)

In [ ]:
local_trends

In [ ]:
trends=local_trends
print(type(trends))
print(list(trends[0].keys()))
print(trends[0]['trends'])

In [ ]:
import json
LocalTrends_json = json.dumps(trends[:3], indent=1)
print(LocalTrends_json)

In [ ]:
book1 = pd.DataFrame({'Name': LocalTrends_json.name, 'URL': LocalTrends_json.url, 'Promoted_Content': LocalTrends_json.promoted_contend, 'Query': LocalTrends_json.query, 'No_Of_Tweets': LocalTrends_json.tweet_volume})


In [ ]:
trends_set = {}
trends_set['world'] = set([trend['name']
                          for trend in world_trends[0]['trends']])

trends_set['us'] = set([trend['name']
                       for trend in us_trends[0]['trends']])

trends_set['Chennai'] = set([trend['name']
                            for trend in local_trends[0]['trends']])

In [ ]:
for loc in ['world', 'us', 'Chennai']:
    print(('-'*10,loc))
    print((','.join(trends_set[loc])))

In [ ]:
print(('='*10, 'intersection of World and US'))
print(trends_set['world'].intersection(trends_set['us']))
print(trends_set['world'].intersection(trends_set['Chennai']))
print(trends_set['us'].intersection(trends_set['Chennai']))

In [ ]:
topic = '#RaniPadmavatiArrivesTomorrow'
number = 100
search_results = twitter_api.search.tweets(q=topic, count=number)
statuses = search_results['statuses']

In [ ]:
len(statuses)

In [ ]:
print((json.dumps(statuses[:2], indent=1)))

In [ ]:
all_text = []
filtered_status = []
for s in status:
    if not s["text"] in all_text:
        filtered_status.append(s)
        all_text.append(s["text"])
statuses = filtered_status     

In [ ]:
len(statuses)

In [ ]:
print((json.dumps(statuses[0], indent=1)))

In [ ]:
status_texts = [ status['text']
              for status in statuses]

screen_name = [ user_mention['screen_name']
              for status in statuses
                 for user_mention in status['entities']['user_mentions']]

hashtags = [ hashtag['text']
           for status in statuses
               for hashtag in status['entities']['hashtags']]

words = [ w
        for t in status_texts
            for w in t.split()]


In [ ]:
print((json.dumps(status_texts[0:5], indent=1)))
print((json.dumps(screen_name[0:5], indent=1)))
print((json.dumps(hashtags[0:5], indent=1)))
print((json.dumps(words[0:5], indent=1)))

In [ ]:
from collections import Counter

for items in [words, screen_name, hashtags]:
    c = Counter(items)
    print(c.most_common()[:10])
    print()

In [ ]:
def prettyprint_counts(label, list_of_tuples):
    print("\n{:^30} | {:^10}".format(label, "Count"))
    print("*"*50)
    for k,v in list_of_tuples:
        print("{:30} | {:>10}".format(k,v))

In [ ]:
for label, data in (('Word', words),
                   ('Screen Name', screen_name),
                   ('Hashtag', hashtags)):
    c = Counter(data)
    prettyprint_counts(label, c.most_common()[:10])

In [ ]:
retweets = [
            
            (status['retweet_count'], 
             status['retweeted_status']['user']['screen_name'], 
             status['text'].replace("\n", "\\"))
    
    for status in statuses
    if 'retweeted_status' in status
             ]
             

In [ ]:
row_template = "{:^7} | {:^15} | {:60}"
def prettyprints_tweets(list_of_tuples):
    print()
    print(row_template.format("Count", "Screen Name", "Text"))
    print("*"*67)
    for count, screen_name, text in list_of_tuples:
        print(row_template.format(count, screen_name, text[:50]))
        if len(text) > 50:
            print(row_template.format("", "", text[50:100]))
            if len(text) > 100:
                print(row_template.format("", "", text[100:]))

In [ ]:
prettyprints_tweets(sorted(retweets, reverse=True)[:5])